In [1]:
import sys
sys.path.append('..')

from janusq.hyqsat import solveByHyqsat, solveByMinisat

# Solving 3-SAT Problem with Hybrid Quantum-Classical SAT solver 

**Author:** Hanyu Zhang \& Siwei Tan

**Date:** 9/4/2024

Based on paper "[QuFEM: Fast and Accurate Quantum Readout Calibration Using the Finite Element Method][1]"

[1]: https://scholar.google.com/scholar_url?url=https://dl.acm.org/doi/abs/10.1145/3613424.3614274%3Fcasa_token%3DffjIB1hQ4ZwAAAAA:8MajDLrDOC74WoeMf7r7AoQ-koxCa4E1TNqQg3GSDz03xUX6XdE3toNTM-YdM_e4rKEusMceJ6BGJg&hl=zh-CN&sa=T&oi=gsb&ct=res&cd=0&d=11146218754516883150&ei=42YSZpPlFL6s6rQPtt6x6Ac&scisig=AFWwaeYaiu2hyx8HUJ_7Buf9Mwom


Quantum readout is the operation that read the information from quantum bits (qubits) into classical bits. Quantum readout noise turns out to be the most significant source of error, which greatly affects the measurement fidelity. In most quantum algorithms, outputs are probability distributions of bit strings, obtained through repeated circuit executions. These bit strings exist in a space represented by $\left \{ 0,1 \right \} ^{N_q}$, where $N_q$ is the number of qubits. The observed probability of a specific bit string reflects its amplitude in the measured quantum superposition state.

In [2]:
# input cnf flie
file_path = "cnf_examples/uf50-01.cnf"
# if verbose
verbose = True
# cpuLim time (s). 0 means infinite
cpuLim = 0
# memLim . 0 means infinite
memLim = 0
# isStrict
strictp = True

In [3]:
# solve by hyqsat
solveByHyqsat(file_path, verb=verbose, cpuLim=cpuLim, memLim=memLim)

PermissionError: [Errno 13] Permission denied: '/home/JanusQ-main/examples/../janusq/hyqsat/./minisat_core'

In [ ]:
# solve by minisat
solveByMinisat(file_path, verb=verbose, cpuLim=cpuLim, memLim=memLim)

2024-04-12 02:02:57,029 - INFO: WARNING: for repeatability, setting FPU to use double precision
2024-04-12 02:02:57,030 - INFO: Original method:
2024-04-12 02:02:57,031 - INFO: /home/jiaxinghui/JanusQ/examples/cnf_examples/uf50-01.cnf
2024-04-12 02:02:57,075 - INFO: ============================[ Problem Statistics ]=============================
2024-04-12 02:02:57,076 - INFO: |                                                                             |
2024-04-12 02:02:57,077 - INFO: |  Number of variables:            50                                         |
2024-04-12 02:02:57,077 - INFO: |  Number of clauses:             218                                         |
2024-04-12 02:02:57,078 - INFO: ============================[ Search Statistics ]==============================
2024-04-12 02:02:57,078 - INFO: | Conflicts |          ORIGINAL         |          LEARNT          | Progress |
2024-04-12 02:02:57,078 - INFO: |           |    Vars  Clauses Literals |    Limit  Clauses L

{'restarts': 1,
 'conflicts': 16,
 'conflict cost': 0.088,
 'decisions': 0,
 'propagations': 0,
 'conflict literals': 65,
 'actual CPU time': 0.044758,
 'this problem time': 0.365,
 'annealing time': 0.0,
 'quantum count': 0,
 'simulate time': 0.0,
 'quantum success number': 0,
 'quantum conflict number': 0,
 'quantum one time solve number': 0,
 'isSatisfiable': True,
 'isSat': True}